In [1]:
import json, requests, re, nltk
from pprint import pprint
from watson_developer_cloud import DiscoveryV1

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
def removePunc(s):
    words = nltk.word_tokenize(s)
    words=[word.lower() for word in words if word.isalpha()]
    sentence = ""
    count = 0
    for word in words:
        if(count == 0):
            sentence = word
        else:
            sentence = sentence + " " + word
        count = count + 1
    return(sentence)

In [15]:
def getIBMCreds():
    credentials = json.load(open('../credentials.json'))
    ibmCreds = credentials['ibm']['discovery']
    return ibmCreds

In [16]:
def getQuery():
    tts('Please state your query. ')
    record_to_file('discoveryQuery.wav')
    query = discoveryStt()
    return(query)

In [17]:
def constructQuery(query):
    query_options = {'natural_language_query': query,
                 'count' : 3,
                 'passages':True}
    return(query_options)

In [18]:
def getQueryResults(discoveryDetails, queryOptions):
    env_id, collections, discovery = discoveryDetails
    queryResults = discovery.query(environment_id=env_id, 
                               collection_id=collections[0]['collection_id'],
                               natural_language_query=queryOptions['natural_language_query'], 
                               passages=queryOptions['passages'], count=queryOptions['count'])
    return(queryResults)

In [19]:
def cleanPassage(passage):
    tagRegEx = re.compile('<.*?>')
    cleanedText = re.sub(tagRegEx, '', passage)
    cleanedText = cleanedText.rstrip()
    cleanedText = cleanedText.replace('\n', '. ')
    return(cleanedText)

In [20]:
def getPassage(queryResults):
    passages = []
    for passage in queryResults['passages']:
        passages.append(passage)
    passageScore = 0
    targetPassages = []
    for passage in passages:
        for key in passage.keys():
            if(passageScore < passage['passage_score']):
                passageScore = passage['passage_score']
                targetPassages.append(passage)
    targetPassage = targetPassages[0]['passage_text']
    cleanedPassage = cleanPassage(targetPassage)
    print("Target passage with highest score is: ")
    print(targetPassage)
    return(cleanedPassage)

In [21]:
def askDiscovery():
    ibmCreds = getIBMCreds()
    discovery = DiscoveryV1(version='2017-11-07',
                            username=ibmCreds['username'],
                            password=ibmCreds['password'])
    env = discovery.list_environments()['environments'][1]
    env_id = env['environment_id']
    collections = discovery.list_collections(env_id)
    collections = collections['collections']
    discoveryDetails = (env_id, collections, discovery)
    query = getQuery()
    queryOptions = constructQuery(query)
    queryResults = getQueryResults(discoveryDetails, queryOptions)
    passage = getPassage(queryResults)
    print("Passage after cleaning is: ")
    print("--------------------***--------------------")
    print(passage)
    return(passage)